In [ ]:
import Pkg; Pkg.activate(@__DIR__)
using Pkg; Pkg.instantiate()
using DifferentialEquations, LinearAlgebra, Statistics, Random
using StatsPlots, Plots, Distributions, KernelDensity, StatsBase, CSV, DataFrames
using LaTeXStrings, Measures

### Reaction Kinetics

In [ ]:
# === Load Experimental Data ===
df = CSV.read("datasetsMA/nitrogenlim.csv", DataFrame)
dfs = hcat(df.Xa, df.Xi, df.N, df.S, df.FG, df.MA)
u0 = dfs[1, :]
t_exp = df.time
T = maximum(t_exp)
tspan = (0.0, T)
dt = 0.01
tsteps = collect(0.0:dt:T)
Nt = length(tsteps)

# === Define ODE Model ===
function f!(du, u, p, t)
    # Unpack state and parameters
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σs, σfg, σp = p
    Xact, Xinact, N, Suc, FruGlu, P = u

    # Ensure non-negative values
    ϵ = 1e-8  # Small positive value to avoid division by zero
    Xact_safe = max(Xact, ϵ)
    Xtot_safe = max(Xact + Xinact, ϵ)
    FruGlu_safe = max(FruGlu, ϵ)
    Suc_safe = max(Suc, ϵ)

    # Algebraic equations
    Xtot = Xact + Xinact
    N_int = 0.08 * N
    ratio = Xinact / Xact_safe
    expo_term = (ratio - ϕ) / χacc

    μ = μmax * FruGlu_safe / (FruGlu_safe + KFG + ϵ) * (N / (N+ KN + ϵ))
    μ2 = μ2max * FruGlu_safe / (FruGlu_safe + KFG2 + ϵ) * (1 - exp(expo_term)) * KIN / (KIN + N + ϵ)
    qsplit = qsplit_max * Suc_safe / (Suc_safe + Ksuc + ϵ)
    qp = qpmax * FruGlu_safe / (FruGlu_safe + KPFG + ϵ) *
         (KIP / (KIP + N_int / Xtot_safe + ϵ)) * KIN / (KIN + N + ϵ)

    du[1] = μ * Xact
    du[2] = μ2 * Xact
    du[3] = - (μ / YXa_N) * Xact
    du[4] = - qsplit * Xact
    du[5] = (qsplit - μ / YXa_S - μ2 / YXi_S - qp / YP_S) * Xact
    du[6] = qp * Xact
end

# === Define Noise Function ===
function noise!(du, u, p, t)
    Xact, Xinact, N, Suc, FruGlu, P = u
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σs, σfg, σp = p
    du[1] = σxa * FruGlu / (FruGlu + (KFG)) * Xact
    du[2] = σxi * FruGlu / (FruGlu + (KFG2)) * Xinact
    du[3] = 0.0
    du[4] = 0.0
    # du[3] = (σxa / YXa_N) * FruGlu / (FruGlu + (KFG)) * Xact
    # du[4] = σs * Suc / (Suc + Ksuc) * Xact
    du[5] = σfg * FruGlu / (FruGlu + Ksuc) * (Xact + Xinact)
    du[6] = σp * FruGlu / (FruGlu + (KPFG)) * Xact
end

# === Parameters ===
params = [
    0.125,  # 1. μmax
    0.147,  # 2. KFG
    3.8e-5,  # 3. KN
    0.531,  # 4. YXa_S
    0.799,  # 5. YXi_S
    9.428,  # 6. YXa_N
    0.508,  # 7. YP_S
    1.56,  # 7. ϕ
    0.3,  # 8. χacc
    0.125,  # 9. μ2max
    1.985,  # 10. qsplit_max
    0.00321,  # 11. Ksuc
    0.095,  # 12. qpmax
    1.5,  # 13. KIP
    1.5e-3,  # 14. KIN
    0.0175,  # 15. KPFG
    3.277,  # 16. KFG2
    0.05,   # 17. σxa
    0.05,   # 18. σxi
    0.05,   # 19. σn
    0.05,   # 20. σs
    0.05,   # 21. σfg
    0.05    # 22. σp
]

# === Solve the ODE ===
odeprob = ODEProblem(f!, u0, tspan, params)
odesol = solve(odeprob, Rosenbrock23(), saveat=tsteps, abstol=1e-8, reltol=1e-6)

# === Extract ODE solution ===
Xa_ode = odesol[1, :]
Xi_ode = odesol[2, :]
N_ode  = odesol[3, :]
Suc_ode = odesol[4, :]
FG_ode = odesol[5, :]
P_ode = odesol[6, :]

@show Nt

### Monte Carlo Simulations

In [ ]:
# === Define the non-negative callback and the SDE Problem ===
function project!(integrator)
    integrator.u .= max.(integrator.u, 0.0)
end
proj_cb = DiscreteCallback((u,t,integrator) -> true, project!; save_positions = (false, false))

sdeprob = SDEProblem(f!, noise!, u0, tspan, params)

# === Simulate the scenarios ===
M = 1000  # Number of simulations

ensemble_prob = EnsembleProblem(sdeprob, prob_func = (prob, i, repeat) -> remake(prob, u0 = u0))
ensemble_sol = solve(ensemble_prob, EM(), EnsembleThreads(), trajectories=M, dt=dt,
                     saveat=tsteps, callback=proj_cb)

# Extract results
Xact_mat = hcat([sol[1, :] for sol in ensemble_sol]...)
Xinact_mat = hcat([sol[2, :] for sol in ensemble_sol]...)
N_mat = hcat([sol[3, :] for sol in ensemble_sol]...)
Suc_mat = hcat([sol[4, :] for sol in ensemble_sol]...)
FruGlu_mat = hcat([sol[5, :] for sol in ensemble_sol]...)
P_mat = hcat([sol[6, :] for sol in ensemble_sol]...)


In [ ]:
# === Plotting ===
scene = 100  # Choose the scene to plot (1 to M)
layout = @layout [a b; c d; e f]
p = plot(layout = layout,
         size = (1200, 800),
         margins = 5mm,
         xlabel = "Time (h)",
         ylabel = "Concentration (g/L)",
         titlefont = font(16),
         tickfont  = font(14),
         legendfont = font(12),
         xlims = (0, maximum(odesol.t)+1))

datasets = [
    (Xinact_mat[:,scene], Xi_ode,  "Xi",  :blue),
    (Xact_mat[:,scene],  Xa_ode,   "Xa",  :red),
    (N_mat[:,scene],     N_ode,    "N",   :green),
    (Suc_mat[:,scene],   Suc_ode,  "Suc", :orange),
    (FruGlu_mat[:,scene],FG_ode,   "FG",  :purple),
    (P_mat[:,scene],     P_ode,    "P",   :brown),
]

for (i, (sde_data, ode_data, name, col)) in enumerate(datasets)
    plot!(p,
          odesol.t, sde_data;
          subplot = i,
          label   = "$name SDE",
          linewidth = 2,
          color   = col)
    plot!(p,
          odesol.t, ode_data;
          subplot = i,
          label     = "$name ODE",
          linestyle = :dash,
          color    = :black,)
end

# define your threshold
FG_thr = 0.05 * (u0[4] + u0[5]) 
hline!(p,
       [FG_thr];
       subplot   = 5,           # target the FruGlu cell
       label     = "Min. threshold",  # legend entry
       linestyle = :dashdot,
       linewidth = 2,
       color     = :black)

display(p)

### Longstaff-Schwartz Initializations

In [ ]:
# === Laguerre basis functions up to degree 3 ===
d = 3  # Degree of the Laguerre polynomial

function laguerre_design_matrix(y::Vector{Float64}, d::Int)
    Φ = zeros(length(y), d + 1)
    for i in 1:length(y)
        Φ[i,1] = 1.0
        if d >= 1
            Φ[i,2] = 1 - y[i]
        end
        if d >= 2
            Φ[i,3] = 1 - 2*y[i] + 0.5*y[i]^2
        end
        if d >= 3
            Φ[i,4] = 1 - 3*y[i] + 1.5*y[i]^2 - (1/6)*y[i]^3
        end
    end
    return Φ
end

basis(x) = laguerre_design_matrix(x, d)

# === Reward function ===
reward(s) = -(s-thresh)^2

# === Initialize vectors and matrices ===
stop_step = fill(Nt, M) # Vector to store the stopping time of each path, currently set to the terminal time step
β_matrix = zeros(Nt, d + 1) # Matrix to store the regression coefficients on every time step
thresh = 0.05 *  (u0[4] + u0[5]) # Threshold for stopping condition: 5% of initial substrates (Suc + FruGlu)

### Longstaff-Schwartz Algorithm

In [ ]:
# === Initialize the value function at the final time step ===
reward_now = reward.(FruGlu_mat)
future_reward = copy(reward_now) 

@show reward_now
@show future_reward

# # === Starting the backward induction ===
# for n in (Nt-1):-1:2

#     # Find alive paths at the current time step (not yet stopped)
#     alive = stop_step .== Nt
#     idx   = findall(alive)
#     if isempty(idx);  continue;  end       # nothing alive → skip

#     # Continuation value via regression 
#     S_k = FruGlu_mat[n, idx]           # current state on alive paths
#     X = basis(S_k)                     # design matrix
#     Y = terminal_V[idx]                # discounted future CF
#     β = X \ Y                          # regression coefficients
#     cont = X * β                       # Ŷ = E[future reward | current state]

#     # Exercise decision
#     exercise_now = V[n, idx] .>= cont
#     stop_now = idx[exercise_now]
#     terminal_V[stop_now] .= V[n, stop_now]
#     stop_step[stop_now] .= n

# end

### Plotting

In [ ]:
# # Store the raw index at which each path exercised
# stop_idx   = copy(stop_step)          # length = M, integer ∈ 1:Nt

# # Convert the index to an actual time
# stop_time  = tsteps[stop_idx]         # same length as stop_idx, Float64

# # Plot the histogram
# histogram(stop_time;
#           bins  = :auto,              # let StatsPlots pick a sensible bin count
#           xlabel = "Stopping time t",
#           ylabel = "Number of paths",
#           title  = "Distribution of optimal stopping times")